In [0]:
import gym
import numpy as np
import random as pr
from gym.envs.registration import register

In [0]:
#강화 학습 환경을 설정
register(
    id="FrozenLake-v1",  #강화 학습 환경의 아이디를 설정
    #gym.envs.toy_text:FrozenLakeEnv : FrozenLake 환경을 설정해서 리턴 하도록 설정
    entry_point="gym.envs.toy_text:FrozenLakeEnv", 
    kwargs={
            "map_name": "4x4",  #FrozenLake 의 줄 칸을 설정 4줄 X4 칸
            "is_slippery":True #미끄러짐 효과 설정 안함
            }
)

In [0]:
#gym.make("FrozenLake-v1"): FrozenLake-v1 환경을 설정해서 env에 리턴
env=gym.make("FrozenLake-v1")

In [0]:
#env.render(): Frozen Lake 게임을 출력
env.render()

In [0]:
#env.reset() : 현재 위지를 s 시장 위치로 설정하고 현재 위치를 리턴
#              시작 위치는 첫번째줄 첫번째 칸이므로 0 이 리턴
position=env.reset()

In [0]:
position

In [0]:
#np.zeros(): 0으로 초기화된 배열을 생성
#np.zeros([16(줄),4(칸)]) : 0으로 초기화된 16줄 4칸의 배열을 생성
Q=np.zeros([16,4])
Q
#Q[0, :] : Q배열의 0번째줄 모든 칸
#np.amax(Q[0, :]) :Q배열의 0번째줄 모든 칸에서 최대값 리턴 1
m=np.amax(Q[0, :])
print("m:",m)
#np.argmax(Q[0, :]): Q배열의 0번째줄 모든 칸에서 최대값이 저장된 인덱스 리턴 2
action=np.argmax(Q[0, :])
print("action:",action)

In [0]:
#pr.choice([0,1,2,3]) : 0,1,2,3 중에서 하나의 수를 랜덤하게 선택해서 리턴
action=pr.choice([0,1,2,3])
print("action:",action)

In [0]:
# Q 학습하기
weight=0.99
learning_rate=0.85
# Goal에 도달 여부를 저장할 리스트
successList = []

for index in range(2000):
    print("="*100)
    print("index:",index)
    position = env.reset() 
    
    # 게임의 reward를 저장할 배열, reward는 0이고 goal에 도달 했을 때만
    totalReward = 0
    while True:
        #env.render(): Frozen Lake 게임을 출력
        env.render()
        print()
        rand_arr = np.random.randn(1, 4) # 난수의 범위는 1 미만, 4개를 생성
        print("rand_arr: ", rand_arr)

        noise = rand_arr / ( index+1 ) # 인덱스가 1인 경우 2로 나눠주는게 되겠졍
        print("noise: ", noise) # 진행 할수록 noise는 점점 작아지게 할겁니다
        print("Q[position, :]: ", Q[position, :])
        print("Q[position, :] + noise: ", Q[position, :] + noise)

        action = np.argmax(Q[position, :] + noise)

        print("action: ", action)
        new_position, reward,done,info = env.step(action)
        totalReward += reward
        print("totalReward:",totalReward,"new_position:",new_position,":reward:",reward,":done:",done,":info:",info)
        
        #(1-learning_rate)*Q[position, action]: 기존 데이터는 1-learning_rate만큼 반영
        #learning_rate*(reward+weight*np.max(Q[position, :])):
        #action을 취했을 때 예상되는 데이터는 learning_rate만큼 반영
        Q[position, action]=(1-learning_rate)*Q[position, action]\
                    + learning_rate*(reward+weight*np.max(Q[new_position, : ]))

        #new_position을 position에 대입
        position=new_position
        #done : 게임 종료 여부를 저장 할 변수 
        #       True : 게임 종료
        #       False : 게임 계속 진행
        if done==True:
            break;

    successList.append([index, totalReward])

    print("Q:",Q)
    print("="*100)

In [0]:
Q

In [0]:
successList

In [0]:
resultList=list(zip(*successList)) # 리스트 안에 저장된 데이터 리턴 열방향 순으로
resultList

In [0]:
sum(list(resultList)[1]) # 성공한 횟수

In [0]:
#학습이 끝났으므로 길을 잘 찾아감

position = env.reset()

while True:
    #env.render(): Frozen Lake 게임을 출력
    env.render()
    print()
    action=np.argmax(Q[position, :])
    
    new_position, reward,done,info = env.step(action)
    #reward+np.max(Q[new_position, :]): 현재 reward+Q배열의 position 번째줄 모든 칸에서 최대값 (np.max(Q[new_position, :]) )
    #을 Q[position,action] 에 대입

    #new_position을 position에 대입
    position=new_position
    #done : 게임 종료 여부를 저장 할 변수 
    #       True : 게임 종료
    #       False : 게임 계속 진행
    if done==True:
        break;

In [0]:
import matplotlib.pyplot as plt

#sum(resultList[1]) : resultList[1] (Reward의 합 - 성공 여부가 저장되어 있음) 의 합
#(성공여부의 전체합 즉 2000번 수행하는데 몇번 성공했는지 합) / 2000
#성공률 계산
print("성공률: ", str(sum(resultList[1]) / len(resultList[1] )))


plt.bar(range(len(resultList[1])), resultList[1], color="blue")